In [1]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json
from geopy.geocoders import Nominatim

from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium

import requests
from bs4 import BeautifulSoup

import geocoder
import time

print('Libraries imported.')

Libraries imported.


In [2]:
# Get addresses of parks for kids in the Seattle area.

with open('park_website.txt') as f:
    park_website = f.read()

# The following is the website used.
# The requests module did not work properly.
# I copied the source code of the website, and saved it in park_website.txt.
kid_parks_url = 'https://www.seattleschild.com/10-of-the-best-seattle-area-parks-for-kids/'

with open('park_website.txt') as f:
    park_website = f.read()

data  = park_website
soup = BeautifulSoup(park_website, 'html.parser')
parks_addr = soup.find_all('address')
parks_nm = soup.find_all('strong')

Parks_Dict = {}

for i, entry1 in enumerate(parks_addr):
    address = entry1.get_text()
    if 'Kenmore' in address:
        continue
    for j, entry2 in enumerate(parks_nm):
        name = entry2.get_text()
        if '(' in name:
            continue
        if i == j:
            Parks_Dict[name] = address

df = pd.DataFrame.from_dict(Parks_Dict, orient='index')
df.reset_index(inplace=True)
df.columns = ['Parks','Address']
df.head()

,Parks,Address
0,Discovery Park,"3801 Discovery Park Blvd, Seattle"
1,Carkeek Park,"950 NW Carkeek Park Rd, Seattle"
2,Seward Park,"5895 Lake Washington Blvd S, Seattle"
3,Volunteer Park,"1247 15th Ave E, Seattle"
4,Green Lake,"7201 E Greenlake Dr N, Seattle"


In [3]:
# The following address was returning a Null result for the geolocator result.
# I removed the row from the dataframe to try again.
locator = Nominatim(user_agent='myGeocoder')
address = df.Address[4]
location = locator.geocode(address)
print(location)

None


In [4]:
df.drop(index = 4, inplace=True)
df.reset_index(inplace=True)
df.head()

,index,Parks,Address
0,0,Discovery Park,"3801 Discovery Park Blvd, Seattle"
1,1,Carkeek Park,"950 NW Carkeek Park Rd, Seattle"
2,2,Seward Park,"5895 Lake Washington Blvd S, Seattle"
3,3,Volunteer Park,"1247 15th Ave E, Seattle"
4,5,Meridian Playground,"4649 Sunnyside Ave N, Seattle"


In [5]:
locator = Nominatim(user_agent='myGeocoder')

lat = []
lon = []
for address in df.Address:
    location = locator.geocode(address)
    lat.append(location.latitude)
    lon.append(location.longitude)

df['Latitude'] = lat
df['Longitude'] = lon
df.head()

,index,Parks,Address,Latitude,Longitude
0,0,Discovery Park,"3801 Discovery Park Blvd, Seattle",47.657771,-122.406287
1,1,Carkeek Park,"950 NW Carkeek Park Rd, Seattle",47.711734,-122.368931
2,2,Seward Park,"5895 Lake Washington Blvd S, Seattle",47.549776,-122.257754
3,3,Volunteer Park,"1247 15th Ave E, Seattle",47.630989,-122.312649
4,5,Meridian Playground,"4649 Sunnyside Ave N, Seattle",47.663467,-122.331504


In [6]:
df.to_csv('parks-location-data.csv')

In [16]:
address = 'Seattle, WA'

geolocator = Nominatim(user_agent="sea-surfer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
map_seattle = folium.Map(location=[latitude, longitude], zoom_start=10, tiles='CartoDB positron')

for lat, lng, park in zip(df['Latitude'], df['Longitude'], df['Parks']):
    label = park
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_seattle)  
    
map_seattle

In [8]:
CLIENT_ID = 'MHHLLZOS144U44BZKQTGKNAM4PX2Z10NOKBOK5A2GRCRMKM4'
CLIENT_SECRET = 'FHXGOYHIHHRJDML5YBJX1GYL4OVJTQTW15JFKKT20RPTOREW'
VERSION = '20180605'
radius = 500
LIMIT = 100

In [9]:
def getNearbyVenues(parks, latitudes, longitudes, radius=500):
    venues_list=[]
    for park, lat, lng in zip(parks, latitudes, longitudes):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            park, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Park', 
                  'Park Latitude', 
                  'Park Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

In [10]:
seattle_venues = getNearbyVenues(df['Parks'], df['Latitude'], df['Longitude'])
seattle_venues.head()

,Park,Park Latitude,Park Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Discovery Park,47.657771,-122.406287,Discovery Park,47.658173,-122.410261,Park
1,Discovery Park,47.657771,-122.406287,Disco Park Playground,47.656142,-122.405245,Playground
2,Discovery Park,47.657771,-122.406287,Metropolitan Market,47.655357,-122.400666,Grocery Store
3,Discovery Park,47.657771,-122.406287,Discovery Park South Parking Lot,47.654653,-122.410255,Trail
4,Discovery Park,47.657771,-122.406287,Foster Wellness,47.658524,-122.403650,Spa


In [11]:
# Removing non-commercial venues from the dataframe
non_comm = ['Park','Playground','Trail',
            'Garden','Outdoor Sculpture',
            'Nature Preserve','Scenic Lookout',
            'Memorial Site','Beach',
            'Dog Run','Harbor / Marina','Bus Station',
            'Pool','Lake','Picnic Area','Amphitheater']
to_drop = []

for index, venue_cat in enumerate(seattle_venues['Venue Category'].iloc):
    if venue_cat in non_comm:
        to_drop.append(index)

seattle_venues.drop(index = to_drop, inplace=True)
seattle_venues.reset_index(inplace=True)
seattle_venues.drop(columns = 'index', inplace=True)
seattle_venues.head()

,Park,Park Latitude,Park Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Discovery Park,47.657771,-122.406287,Metropolitan Market,47.655357,-122.400666,Grocery Store
1,Discovery Park,47.657771,-122.406287,Foster Wellness,47.658524,-122.403650,Spa
2,Discovery Park,47.657771,-122.406287,Shell,47.659472,-122.400436,Gas Station
3,Discovery Park,47.657771,-122.406287,Center for Ideal Weight Loss,47.661482,-122.409464,Massage Studio
4,Carkeek Park,47.711734,-122.368931,A Divine Taste Catering,47.714389,-122.373689,Food


In [12]:
seattle_venues.to_csv('venues-near-parks.csv')

In [13]:
food_bev = ['Food','Restaurant','Pizza','Burger','Cupcake','Pub','Bar']
food_bev_list = []

for index, venue_cat in enumerate(seattle_venues['Venue Category'].iloc):
    venue_cat_words = venue_cat.split()
    for word in venue_cat_words:
        if word in food_bev:
            food_bev_list.append(1)
            break
        else:
            food_bev_list.append(0)
            break

'''
I am using 1 and 0 instead of 'Yes' and 'No' to answer 
whether the venue is a food and beverage selling place, 
such as a restaurant, a coffee shop, or a bar.
'''

seattle_venues['Food or Beverage Place'] = food_bev_list
seattle_venues.head()

,Park,Park Latitude,Park Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Food or Beverage Place
0,Discovery Park,47.657771,-122.406287,Metropolitan Market,47.655357,-122.400666,Grocery Store,0
1,Discovery Park,47.657771,-122.406287,Foster Wellness,47.658524,-122.403650,Spa,0
2,Discovery Park,47.657771,-122.406287,Shell,47.659472,-122.400436,Gas Station,0
3,Discovery Park,47.657771,-122.406287,Center for Ideal Weight Loss,47.661482,-122.409464,Massage Studio,0
4,Carkeek Park,47.711734,-122.368931,A Divine Taste Catering,47.714389,-122.373689,Food,1


In [17]:
for lat, lng, fb, venue in zip(seattle_venues['Venue Latitude'], seattle_venues['Venue Longitude'], seattle_venues['Food or Beverage Place'], seattle_venues['Venue']):
    label = venue
    label = folium.Popup(label, parse_html=True)
    if fb == 1:
        col = '#5E2BFF'
    else:
        col = '#DB5461'
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=col,
        fill=True,
        fill_color=col,
        fill_opacity=0.7,
        parse_html=False).add_to(map_seattle)

    
map_seattle

In [28]:
lng = '-122.3315041'
lat = '47.6634669'

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    lng, 
    radius, 
    LIMIT)
results = requests.get(url).json()["response"]['groups'][0]['items']

print([v for v in results][])

[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '57dde06c498e779b8196ab42', 'name': "Ezell's Famous Chicken", 'location': {'address': '2300 N 45th St', 'crossStreet': 'Corliss Ave N', 'lat': 47.66150944254662, 'lng': -122.33084751714829, 'labeledLatLngs': [{'label': 'display', 'lat': 47.66150944254662, 'lng': -122.33084751714829}], 'distance': 223, 'postalCode': '98103', 'cc': 'US', 'city': 'Seattle', 'state': 'WA', 'country': 'United States', 'formattedAddress': ['2300 N 45th St (Corliss Ave N)', 'Seattle, WA 98103', 'United States']}, 'categories': [{'id': '4d4ae6fc7a7b7dea34424761', 'name': 'Fried Chicken Joint', 'pluralName': 'Fried Chicken Joints', 'shortName': 'Fried Chicken', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/friedchicken_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}}, 'referralId': 'e-0-57dde06c498e779b8196ab42-0'},